In [4]:
import pinecone
from tqdm import tqdm
import pandas as pd

c:\Users\dattad\Anaconda3\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [5]:
pinecone.init(api_key="", environment="")


In [6]:
pinecone.list_indexes()

['pinecone-index']

In [5]:
import os
import pandas as pd
os.getcwd()

'c:\\Users\\dattad\\Desktop\\pinecone-poc'

In [6]:
data = pd.read_csv("data/NYTFB-280K/train.txt",sep = "\t",header=1,names = ["meta-1","meta-2","NE1","NE2","relation","text"])

C:\Users\dattad\AppData\Local\Temp\ipykernel_10644\919608044.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data/NYTFB-280K/train.txt",sep = "\t",header=1,names = ["meta-1","meta-2","NE1","NE2","relation","text"])


In [ ]:
dropped_nas = data.dropna()
dropped_nas["relation"]

In [ ]:
len(dropped_nas["relation"].unique())

In [11]:
def clean_text(text):
    text = text.replace("-lrb-","").replace("-rrb-","").replace("###END###","")
    return text.strip()

In [ ]:
dropped_nas['cleaned_text'] = dropped_nas['text'].apply(clean_text)

In [14]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [15]:
import numpy as np
from collections import Counter

In [ ]:
def reduce_dataset(df,category,MIN_SAMPLE_PER_GROUP,TARGET_SAMPLES):

    small_groups = [group for _, group in df.groupby(category) if len(group) < MIN_SAMPLE_PER_GROUP]
    large_groups = [group for _, group in df.groupby(category) if len(group) >= MIN_SAMPLE_PER_GROUP]
    print(len(small_groups))
    print(len(large_groups))

    resampled_df = pd.concat(small_groups)
    remaining_samples = TARGET_SAMPLES - len(resampled_df)
    if remaining_samples > 0:
        samples_per_large_group = remaining_samples // len(large_groups)
        print(samples_per_large_group)
        for group in large_groups:
            print("--->",len(group))
            sampled_group = group.sample(n=samples_per_large_group, replace=False)
            resampled_df = pd.concat([resampled_df, sampled_group])

    resampled_df = resampled_df.reset_index(drop=True)
    return resampled_df

reduced_dataset = reduce_dataset(dropped_nas,"relation",400,10000)


In [ ]:
sentences = reduced_dataset["cleaned_text"].tolist()

embeddings= np.zeros((len(sentences),model.get_sentence_embedding_dimension()))

for i in tqdm(range(len(sentences)),desc = "Encoding Sentences"):
    embeddings[i] = model.encode([sentences[i]])

reduced_dataset["embedding"] = embeddings.tolist()


In [ ]:
reduced_dataset.to_csv("reduced_dataset.csv")

UPSERTING TO PINECONE

In [ ]:
# index_name = "pinecone-index"
# if index_name in pinecone.list_indexes():
#     pinecone.delete_index(index_name)

In [ ]:
reduced_dataset.reset_index(inplace=True)

In [ ]:
reduced_dataset.columns

Index(['index', 'meta-1', 'meta-2', 'NE1', 'NE2', 'relation', 'text',
       'cleaned_text', 'embedding'],
      dtype='object')

In [ ]:
def create_metadata(df):
    df["metadata"] = df.apply(lambda x: {"NE1":x["NE1"],"NE2":x["NE2"],"relation":x["relation"] },axis = 1)
    return df

In [ ]:
reduced_dataset = create_metadata(reduced_dataset)

CREATING PINECONE UPSERT VECTORS

In [ ]:
def create_vectors(df):
    df["pinecone-vectors"] = df.apply(lambda x: (str(x["index"]),x["embedding"],x["metadata"]),axis = 1)
    return df

reduced_dataset = create_vectors(reduced_dataset)

    

In [ ]:
upsert_vectors = reduced_dataset["pinecone-vectors"].to_list()

In [ ]:
index_name = 'pinecone-index'

In [ ]:
# pinecone.create_index(index_name,dimension=384,metric="cosine")

In [ ]:
index = pinecone.Index(index_name)

UPSERT TO PINECONE

In [ ]:
#Upsert to the index with chunk size 500

vector_num = 500

for i in range(0,len(upsert_vectors),vector_num):
    chunk = upsert_vectors[i:i+vector_num]
    upsert_response = index.upsert(chunk)

USING PINECONE FOR FINDING SIMILARITY

query_response

In [ ]:
upsert_vectors[0][2]

{'NE1': 'wdet', 'NE2': 'detroit', 'relation': '/broadcast/content/location'}

In [ ]:
from collections import defaultdict

In [67]:
batch_size =100
top_k = 15
tuple_batches = [upsert_vectors[i:i+batch_size] for i in range(0,10000,batch_size)]

In [9]:
index_name = 'pinecone-index'
index = pinecone.Index(index_name)

In [18]:
results = index.query(top_k = 25,vector=upsert_vectors[0][1],include_metadata=True)

In [27]:
from collections import Counter

In [29]:
dict(Counter([result["metadata"]["relation"] for result in results["matches"]]).most_common())

{'/business/company/major_shareholders': 10,
 '/business/company_shareholder/major_shareholder_of': 4,
 '/business/company/founders': 4,
 '/broadcast/content/location': 2,
 '/people/person/place_of_birth': 2,
 '/business/company/place_founded': 2,
 '/people/person/place_lived': 1}

In [36]:
text_liste = [result["metadata"]["relation"] for result in results["matches"]]
text = "\n".join(text_liste)
       


In [34]:
text

'/broadcast/content/location\n/broadcast/content/location\n/people/person/place_of_birth\n/business/company/place_founded\n/business/company/place_founded\n/business/company/major_shareholders\n/business/company_shareholder/major_shareholder_of\n/business/company/major_shareholders\n/business/company/founders\n/people/person/place_lived\n/business/company/founders\n/business/company/founders\n/business/company/major_shareholders\n/business/company/major_shareholders\n/business/company_shareholder/major_shareholder_of\n/business/company/founders\n/business/company_shareholder/major_shareholder_of\n/business/company/major_shareholders\n/business/company/major_shareholders\n/business/company_shareholder/major_shareholder_of\n/business/company/major_shareholders\n/business/company/major_shareholders\n/people/person/place_of_birth\n/business/company/major_shareholders\n/business/company/major_shareholders'

In [37]:
text2

'/broadcast/content/location\n/broadcast/content/location\n/people/person/place_of_birth\n/business/company/place_founded\n/business/company/place_founded\n/business/company/major_shareholders\n/business/company_shareholder/major_shareholder_of\n/business/company/major_shareholders\n/business/company/founders\n/people/person/place_lived\n/business/company/founders\n/business/company/founders\n/business/company/major_shareholders\n/business/company/major_shareholders\n/business/company_shareholder/major_shareholder_of\n/business/company/founders\n/business/company_shareholder/major_shareholder_of\n/business/company/major_shareholders\n/business/company/major_shareholders\n/business/company_shareholder/major_shareholder_of\n/business/company/major_shareholders\n/business/company/major_shareholders\n/people/person/place_of_birth\n/business/company/major_shareholders\n/business/company/major_shareholders'

In [117]:

neighbours = defaultdict(set)
threshold = 0.55
index = pinecone.Index(index_name)

for batch in tqdm(tuple_batches):
    ids,vectors,_ = zip(*batch)
    results = index.query(top_k = 25,queries=vectors)
    for i,element in enumerate(results['results']):
        # if element["score"] > threshold:
        neighbours[str(ids[i])].update([res['id'] for res in element["matches"] if ((res['score'] > threshold) and (str(res['id']) != str(ids[i]) ))])


100%|██████████| 100/100 [06:40<00:00,  4.01s/it]


In [ ]:
len(neighbours)

10000

In [ ]:
neighbours_dict = {}
neighbours_dict = dict(neighbours)

In [ ]:
import pickle

with open('neigbours.pkl','wb') as f:
    pickle.dump(dict(neighbours),f)

with open('upsert_vectors.pkl','wb') as f:
    pickle.dump(upsert_vectors,f)

with open('reduced_dataset.pkl','wb') as f:
    pickle.dump(reduced_dataset,f)

In [66]:
from langchain.embeddings.openai import OpenAIEmbeddings
import openai

import os

os.environ["OPENAI_API_TYPE"] = ""
os.environ["OPENAI_API_BASE"] = ""
os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_VERSION"] = ""
EMBEDDING_DEPLOYED_NAME = ""
EMBEDDING_MODEL_NAME = ""
openai.api_key = ""

def get_embeddings(text):
    embeddings =  OpenAIEmbeddings(
    deployment=EMBEDDING_DEPLOYED_NAME,
    model=EMBEDDING_MODEL_NAME)
    response = openai.Embedding.create(
            input=text,
            engine=EMBEDDING_DEPLOYED_NAME,
            deployment_id=EMBEDDING_DEPLOYED_NAME
        
)
    return response['data'][0]['embedding']
    # return embeddings.embed_query(text)


# get_embeddings("hello")

# SUMMARIZATION

# SUMMARIZE NEIGBHOURHOOLD
THE VECTOR INDEX, EMBEDDING AND THE PINECONE INDEX ARE TO BE PASSED INTO THIS FUNCTION TO GET THE SUMMARY AND THE RELATIONS DICTIONARY

In [51]:
import pickle

with open('upsert_vectors.pkl','rb') as f:
    upsert_vectors = pickle.load(f)
with open('reduced_dataset.pkl','rb') as f:
    reduced_dataset = pickle.load(f)
with open('neigbours.pkl','rb') as f:
    neighbours = pickle.load(f)


In [47]:
def summarize_neighbourhood(_id,_embed,_index,_engine = 'Davinci-1',_relations = 'relations',_sentence = 'sentence',_k = 15,_random_state = 42,_max_tokens = 100,prompt = 'What do the following texts have in common? Is there any common theme in these texts?'):
    
    if _index: 
        nns = _index.query(top_k=_k,
                               vector = _embed,include_metadata=True)
        
        metadata_dict = dict(Counter([result["metadata"][_relations] for result in results["matches"]]).most_common())
        text_liste = [result["metadata"][_sentence] for result in results["matches"]]
        
        text = "\n".join(text_liste)

        response = openai.Completion.create(
            engine= _engine,
            prompt=f'{prompt} \n"""\n{text}\n"""\n\n Theme:',
            temperature=0,
            max_tokens=_max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
        )
        summary = "Summary:" + response["choices"][0]["text"].replace("\n", "")
    else:
        summary = 'Pinecone index not found'
    
    return summary,metadata_dict


In [64]:
index

In [48]:
summarize_neighbourhood(_id = upsert_vectors[0][0],_embed = upsert_vectors[0][1],_index = index,_engine = 'Davinci-1',_relations = 'relation',_sentence = 'relation',_k = 15,_random_state = 42,_max_tokens = 100,prompt = 'What do the following texts have in common? Is there any common theme in these texts?')

("Summary: Locations and People Related to Companies The common theme in these texts is locations and people related to companies. This includes the location of a company's founding, the place of birth of its founders, the major shareholders, and the places where the founders have lived.",
 {'/business/company/major_shareholders': 10,
  '/business/company_shareholder/major_shareholder_of': 4,
  '/business/company/founders': 4,
  '/broadcast/content/location': 2,
  '/people/person/place_of_birth': 2,
  '/business/company/place_founded': 2,
  '/people/person/place_lived': 1})

In [53]:
def dfs(_node, _cluster_id,_clusters,_neighbours):
    stack = [_node]
    while stack:
        _node = stack.pop()
        if _node not in _clusters:
            _clusters[_node] = _cluster_id
            stack.extend(_neighbours[_node] - set(_clusters.keys()))
    return _clusters

In [52]:
def run_clustering(_neighbours):
    clusters = {}
    cluster_id = 0

    for index in _neighbours.keys():
        if index not in clusters:
            clusters = dfs(index, cluster_id,clusters,_neighbours)
            cluster_id += 1

    return clusters

In [54]:
clusters = run_clustering(neighbours)

In [55]:
from collections import defaultdict
df = reduced_dataset.copy()
df["index"] = df["index"].astype(str)
df['cluster'] = df["index"].map(clusters)


In [56]:
umap_df = df.copy()

def all_vs_topklabelling(df,column,k):
    list_tuples = Counter(df[column]).most_common()[:k]
    top_k_labels = [item[0] for item in list_tuples]
    df["cluster_topk"] = df[column].apply(lambda x: x if x in top_k_labels else "-1")
    return df

umap_df = all_vs_topklabelling(umap_df,"cluster",10)

    


# SUMMARIZE CLUSTERS

THE BELOW FUNCTION IS FOR SUMMARIZING CLUSTERS (SAMPLES_PER_CLUSTER = 15)

In [61]:
def summarize_clusters(_dataframe,_text_column_name,_cluster_column_name,_cluster_labels,_engine = 'Davinci-1',_samples_per_cluster = 15,_random_state = 42,_max_tokens = 100,prompt = 'What do the following texts have in common? Is there any common theme in these texts?'):
    
    output_dict = {"Cluster_Num" : [],"Summary" : []}
    for i in _cluster_labels:
        text = "\n".join(
            _dataframe[_dataframe[_cluster_column_name] == i][_text_column_name]
            .sample(_samples_per_cluster, random_state = _random_state)
            .values
        )
        response = openai.Completion.create(
            engine= _engine,
            prompt=f'{prompt} \n"""\n{text}\n"""\n\n Theme:',
            temperature=0,
            max_tokens=_max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
        )
        output_dict["Cluster_Num"].append(i)
        output_dict["Summary"].append(response["choices"][0]["text"].replace("\n", ""))
    return pd.DataFrame(output_dict)


DATAFRAME EXPECTED FOR the SUMMARIZE_CLUSTERS FUNCTION

In [60]:
umap_df[["cleaned_text","cluster_topk"]]

,cleaned_text,cluster_topk
0,"mr. coleman , the former deputy director , is ...",-1
1,it looked like that script was being followed ...,-1
2,"from a business standpoint , the cardinals did...",-1
3,"from a business standpoint , the cardinals did...",-1
4,"from a business standpoint , the cardinals did...",-1
...,...,...
9995,a9 new hurdle for iraq constitution at least t...,-1
9996,chabot in cincinnati will have a little more o...,-1
9997,gates village voice best of 2004 through wed...,-1
9998,"in his presentation in tokyo yesterday , ken_k...",-1


In [62]:
cluster_labels = umap_df["cluster_topk"].unique().tolist()[1:]

RESPONSE OF SUMMARIZE_CLUSTERS

In [63]:
summarize_clusters(umap_df,"cleaned_text","cluster_topk",cluster_labels,_samples_per_cluster = 15)

,Cluster_Num,Summary
0,33,International Relations
1,81,The common theme in these texts is the compan...
2,86,The common theme in these texts is the relati...
3,99,Politics and Government The common theme in t...
4,265,The common theme in these texts is the politi...
5,276,Death and memorials
6,510,The texts have a common theme of the National...
7,518,Football The common theme in these texts is f...
8,632,The common theme in these texts is the fashio...
9,3486,The common theme in these texts is the story ...
